In [2]:
import pyspark
import os
import shutil
root_folder = "/home/trungdc/unimelb/MAST30024/asm/project 1/"
data_dir = os.path.join(root_folder, "Data/Trip")


import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import *

In [2]:
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
spark = SparkSession.builder.getOrCreate()
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)

Download green trip record (Month: 2,4,6,8,10,12) to folder: Data/Green

In [4]:
sdf = spark.read.format("csv").option("header", "true").load(os.path.join(data_dir,"Green","*.csv"))
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
sdf.createOrReplaceTempView("trip")
sdf.limit(5)

VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
2,2018-02-01 00:39:38,2018-02-01 00:39:41,N,5,97,65,1,.00,20,0,0,3,0,null,0,23,1,2
2,2018-02-01 00:58:28,2018-02-01 01:05:35,N,1,256,80,5,1.60,7.5,0.5,0.5,0.88,0,null,0.3,9.68,1,1
2,2018-02-01 00:56:05,2018-02-01 01:18:54,N,1,25,95,1,9.60,28.5,0.5,0.5,5.96,0,null,0.3,35.76,1,1
2,2018-02-01 00:12:40,2018-02-01 00:15:50,N,1,61,61,1,.73,4.5,0.5,0.5,0,0,null,0.3,5.8,2,1
2,2018-02-01 00:45:18,2018-02-01 00:51:56,N,1,65,17,2,1.87,8,0.5,0.5,0,0,null,0.3,9.3,2,1


In [5]:
sql_query = """
SELECT *
FROM trip
ORDER BY "tpep_pickup_datetime"
"""
out = spark.sql(sql_query)
out.head()

Row(VendorID='2', lpep_pickup_datetime='2018-02-01 00:39:38', lpep_dropoff_datetime='2018-02-01 00:39:41', store_and_fwd_flag='N', RatecodeID='5', PULocationID='97', DOLocationID='65', passenger_count='1', trip_distance='.00', fare_amount='20', extra='0', mta_tax='0', tip_amount='3', tolls_amount='0', ehail_fee=None, improvement_surcharge='0', total_amount='23', payment_type='1', trip_type='2')

In [7]:
print((sdf.count(), len(sdf.columns)))

(7309493, 19)


In [8]:
out.repartition(1).write.csv(os.path.join(data_dir,"green.csv"), header=True)

In [3]:
for file in os.listdir(os.path.join(data_dir, "green.csv")):
    if file.endswith(".csv"):
        os.rename(os.path.join(data_dir, "green.csv", file), os.path.join(data_dir, "green_concat.csv"))
shutil.rmtree(os.path.join(data_dir, "green.csv"))